# MMAI 894 - Exercise 2
## Convolutional artificial neural network : Image classification
The goal of this excercise is to build a convolutional neural network using the tensorflow/keras library. We will be using the MNIST dataset.
Submission instructions:

- You cannot edit this notebook directly. Save a copy to your drive, and make sure to identify yourself in the title using name and student number
- Do not insert new cells before the final one (titled "Further exploration") 
- Verify that your notebook can _restart and run all_. 
- Select File -> Download as .py (important! not as ipynb)
- Rename the file: `studentID_lastname_firstname_ex2.py`
- The mark will be assessed on the implementation of the functions with #TODO
- **Do not change anything outside the functions**  unless in the further exploration section
- As you are encouraged to explore the network configuration, 20% of the mark is based on final accuracy. 
- Note: You do not have to answer the questions in thie notebook as part of your submission. They are meant to guide you.

- You should not need to use any additional libraries other than the ones listed below. You may want to import additional modules from those libraries, however.

In [ ]:
# Import modules
# Add modules as needed
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# For windows laptops add following 2 lines:
import matplotlib
matplotlib.use('agg')

import matplotlib.pyplot as plt
import tensorflow as tf

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Dropout, Conv2D,Flatten, MaxPooling2D
from tensorflow.keras.optimizers import SGD
import numpy as np
import pandas as pd
import argparse

%matplotlib inline


### Data preparation

#### Import data

In [ ]:
def load_data():
    # Import MNIST dataset from openml
    dataset = fetch_openml('mnist_784', version=1, data_home=None, as_frame=False)

    # Data preparation
    raw_X = dataset['data']
    raw_Y = dataset['target']
    return raw_X, raw_Y

raw_X, raw_Y = load_data()

## Consider the following
- Same as excercise 1
- what shape should x be for a convolutional network?

In [ ]:
def clean_data(raw_X, raw_Y):
    # TODO: clean and QA raw_X and raw_Y
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION

    raw_X = (raw_X/255)

    cleaned_X = tf.keras.utils.normalize(raw_X, axis=-1, order=1)
    cleaned_Y = tf.keras.utils.to_categorical(raw_Y)
    
    return cleaned_X, cleaned_Y

cleaned_X, cleaned_Y = clean_data(raw_X, raw_Y)


#### Data split

- Split your data into a train set (50%), validation set (20%) and a test set (30%). You can use scikit-learn's train_test_split function.

In [ ]:
def split_data(cleaned_X, cleaned_Y):
    # TODO: split the data
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION
    train_ratio = 0.50
    validation_ratio = 0.20
    test_ratio = 0.30
    
    X_train, X_test, Y_train, Y_test = train_test_split(cleaned_X, cleaned_Y, test_size = 1 - train_ratio)
    X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=test_ratio/(test_ratio + validation_ratio))

    
    return X_val, X_test, X_train, Y_val, Y_test, Y_train

X_val, X_test, X_train, Y_val, Y_test, Y_train = split_data(cleaned_X, cleaned_Y)

### Model

#### Neural network structure

This time, the exact model architecture is left to you to explore.  
Keep the number of parameters below 2,000,000

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
X_val = X_val.reshape(-1, 28, 28, 1)
input_shape = (28, 28, 1)

def build_model():
    # TODO: build the model, 
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION
    model = models.Sequential()
    model.add(Conv2D(120,
    kernel_size=(3,3),
    activation = 'relu',
    input_shape=(28,28,1),
    padding='same'))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(10, activation ='softmax'))  
    
    return model

def compile_model(model):
    # TODO: compile the model
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy', 
              metrics=['accuracy']
    )

    return model

def train_model(model, X_train, Y_train, X_val, Y_val):
    # TODO: train the model
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION

    history = model.fit(
    x=X_train,
    y=Y_train,
    batch_size=128,
    epochs=12,
    verbose="auto",
    validation_data=(X_val, Y_val)
    )

    return model, history


def eval_model(model, X_test, Y_test):
    # TODO: evaluate the model
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION

    print("Evaluate on test data")
    results = model.evaluate(X_test, Y_test, batch_size=128)
    print("test loss, test acc:", results)
    test_loss = results[0]
    test_accuracy = results[1]

    # Generate predictions (probabilities -- the output of the last layer)
    # on new data using `predict`
    print("Generate predictions for 3 samples")
    predictions = model.predict(X_test[:3])
    print("predictions shape:", predictions.shape)
  

    return test_loss, test_accuracy



    return test_loss, test_accuracy



In [ ]:
## You may use this space (and add additional cells for exploration)

model = build_model()
model = compile_model(model)
model, history = train_model(model, X_train, Y_train, X_val, Y_val)
test_loss, test_accuracy = eval_model(model, X_test, Y_test)

Epoch 1/12
274/274 [==============================] - 123s 448ms/step - loss: 0.8557 - accuracy: 0.7289 - val_loss: 0.3469 - val_accuracy: 0.8954
Epoch 2/12
274/274 [==============================] - 120s 437ms/step - loss: 0.2683 - accuracy: 0.9202 - val_loss: 0.1978 - val_accuracy: 0.9410
Epoch 3/12
274/274 [==============================] - 123s 448ms/step - loss: 0.1704 - accuracy: 0.9485 - val_loss: 0.1372 - val_accuracy: 0.9595
Epoch 4/12
274/274 [==============================] - 120s 437ms/step - loss: 0.1211 - accuracy: 0.9628 - val_loss: 0.1167 - val_accuracy: 0.9649
Epoch 5/12
274/274 [==============================] - 120s 438ms/step - loss: 0.0954 - accuracy: 0.9708 - val_loss: 0.0970 - val_accuracy: 0.9707
Epoch 6/12
274/274 [==============================] - 121s 442ms/step - loss: 0.0814 - accuracy: 0.9745 - val_loss: 0.0916 - val_accuracy: 0.9723
Epoch 7/12
274/274 [==============================] - 120s 438ms/step - loss: 0.0667 - accuracy: 0.9789 - val_loss: 0.0780 -